<a href="https://colab.research.google.com/github/zwzzhong/Natural-Language-Processing/blob/master/NLP_with_GCP_and_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Natural Language Processing**

## Ingest

In [0]:
!pip -q install boto3

In [0]:
import pandas as pd 
import json 
import os #to set credentials to enviroment 

#AWS Comprehend
import boto3

# Imports the Google Cloud client library for GCP Natural Language API
from google.cloud import language_v1
from google.cloud.language_v1 import enums
import six

In [0]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
df = pd.read_csv("/content/gdrive/My Drive/Amazon_Review.csv")

df = df[['reviews.rating','reviews.text']]

df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,reviews.rating,reviews.text
0,5.0,This product so far has not disappointed. My c...
1,5.0,great for beginner or experienced person. Boug...
2,5.0,Inexpensive tablet for him to use and learn on...
3,4.0,I've had my Fire HD 8 two weeks now and I love...
4,5.0,I bought this for my grand daughter when she c...


In [0]:
df1 = df[df['reviews.rating'] == 1].head(25)
df2 = df[df['reviews.rating'] == 2].head(25)
df3 = df[df['reviews.rating'] == 3].head(25)
df4 = df[df['reviews.rating'] == 4].head(25)
df5 = df[df['reviews.rating'] == 5].head(25)


df = pd.concat((df1, df2, df3, df4, df5))
df['reviews'] = df['reviews.text']
df = df.drop('reviews.text', axis=1)

In [0]:
df.loc[ df["reviews.rating"] < 3, 'Sentiment' ] = 'NEGATIVE'
df.loc[ df["reviews.rating"] > 3, 'Sentiment' ] = 'POSITIVE'
df.loc[ df["reviews.rating"] == 3, 'Sentiment' ] = 'NEUTRAL'

## AWS 

**Install boto3**


In [0]:
import os;os.listdir("/content/gdrive/My Drive/Colab Notebooks/Credentials")

['rootkey.csv', 'credentials.csv', 'config.txt', 'credentials.txt']

**Create API Config**

In [0]:
!mkdir -p ~/.aws &&\
  cat /content/gdrive/My\ Drive/Colab\ Notebooks/Credentials/credentials.txt> ~/.aws/credentials

!cd ~/.aws &&\
  cat /content/gdrive/My\ Drive/Colab\ Notebooks/Credentials/config.txt > ~/.aws/config

**Call API for AWS sentiment analysis**

In [0]:
#Test AWS comprehend
comprehend = boto3.client(service_name='comprehend', region_name="us-east-2")
text = "There is smoke in San Francisco"
comprehend.detect_sentiment(Text=text, LanguageCode='en')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '163',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 15 Mar 2019 01:48:59 GMT',
   'x-amzn-requestid': '80615f8f-46c4-11e9-8d1d-d5aa0add79c7'},
  'HTTPStatusCode': 200,
  'RequestId': '80615f8f-46c4-11e9-8d1d-d5aa0add79c7',
  'RetryAttempts': 0},
 'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Mixed': 0.0032753823325037956,
  'Negative': 0.05816865712404251,
  'Neutral': 0.929866373538971,
  'Positive': 0.008689661510288715}}

In [0]:
comprehend = boto3.client(service_name='comprehend', region_name="us-east-2")

def get_sentiment(text):
  try:
    res = comprehend.detect_sentiment(Text=text, LanguageCode='en')["Sentiment"]
    return res
  except: 
    return None

In [0]:
df["reviews"].astype(str)

df["AWS_sentiment"] = df["reviews"].apply(lambda x: get_sentiment(x))

In [0]:
df.head()

,reviews.rating,reviews,Sentiment,AWS_sentiment
126,1.0,i Bought this around black friday for $60 hopi...,NEGATIVE,NEGATIVE
169,1.0,I bought this tablet for my 4 year old daughte...,NEGATIVE,NEGATIVE
241,1.0,Didn't want to go to the sites amazon wanted m...,NEGATIVE,NEGATIVE
297,1.0,This Kindle is so overloaded with apps that I ...,NEGATIVE,NEGATIVE
310,1.0,I liked it at first except the browser is kind...,NEGATIVE,NEGATIVE


## GCP 

In [0]:
!pip install --upgrade -q google-cloud-language

    100% |████████████████████████████████| 71kB 4.3MB/s 


In [0]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/gdrive/My Drive/flowersapi.json"

**Call API for GCP sentiment analysis**

In [0]:
def gcp_analyze_sentiment(content):
    client = language_v1.LanguageServiceClient()
    if isinstance(content, six.binary_type):
        content = content.decode('utf-8')
    type_ = enums.Document.Type.PLAIN_TEXT
    document = {'type': type_, 'content': content}
    response = client.analyze_sentiment(document)
    sentiment = response.document_sentiment.score
    return(sentiment)

In [0]:
df["GCP_sentiment_score"] = df["reviews"].apply(lambda x: gcp_analyze_sentiment(x))

In [0]:
df.head()

,reviews.rating,reviews,Sentiment,AWS_sentiment,GCP_sentiment_score
126,1.0,i Bought this around black friday for $60 hopi...,NEGATIVE,NEGATIVE,-0.3
169,1.0,I bought this tablet for my 4 year old daughte...,NEGATIVE,NEGATIVE,-0.3
241,1.0,Didn't want to go to the sites amazon wanted m...,NEGATIVE,NEGATIVE,-0.7
297,1.0,This Kindle is so overloaded with apps that I ...,NEGATIVE,NEGATIVE,-0.3
310,1.0,I liked it at first except the browser is kind...,NEGATIVE,NEGATIVE,-0.2


**Transform into same scale**

In [0]:
df.loc[ df["GCP_sentiment_score"] < 0.5, 'GCP_sentiment' ] = 'NEGATIVE'
df.loc[ df["GCP_sentiment_score"] > 0.5, 'GCP_sentiment' ] = 'POSITIVE'
df.loc[ df["GCP_sentiment_score"] == 0.5, 'GCP_sentiment' ] = 'NEUTRAL'

In [0]:
df.head()

,reviews.rating,reviews,Sentiment,AWS_sentiment,GCP_sentiment_score,GCP_sentiment
126,1.0,i Bought this around black friday for $60 hopi...,NEGATIVE,NEGATIVE,-0.3,NEGATIVE
169,1.0,I bought this tablet for my 4 year old daughte...,NEGATIVE,NEGATIVE,-0.3,NEGATIVE
241,1.0,Didn't want to go to the sites amazon wanted m...,NEGATIVE,NEGATIVE,-0.7,NEGATIVE
297,1.0,This Kindle is so overloaded with apps that I ...,NEGATIVE,NEGATIVE,-0.3,NEGATIVE
310,1.0,I liked it at first except the browser is kind...,NEGATIVE,NEGATIVE,-0.2,NEGATIVE


## **Evaluate accuracy**

**GCP VS AWS**

In [0]:
df.loc[df["GCP_sentiment"] == df["AWS_sentiment"], "gcp_euqal_aws"] = 1
euqual_gcp_aws = df["gcp_euqal_aws"].sum()
total = len(df)
identity_ratio_gcp_aws = euqual_gcp_aws/total

identity_ratio_gcp_aws

0.712

**GCP VS ORIGIN**

In [0]:
df.loc[df["GCP_sentiment"] == df["Sentiment"], "gcp_euqal_origin"] = 1
equal_gcp_origin = df["gcp_euqal_origin"].sum()
total = len(df)
identity_ratio_gcp_origin = equal_gcp_origin/total

identity_ratio_gcp_origin

0.704

**AWS VS ORIGIN**

In [0]:
df.loc[df["AWS_sentiment"] == df["Sentiment"], "aws_euqal_origin"] = 1
equal_aws_origin = df["aws_euqal_origin"].sum()
total = len(df)
identity_ratio_aws_origin = equal_aws_origin/total

identity_ratio_aws_origin

0.672